In [ ]:
import numpy as np
from PIL import Image
import os
from os.path import isdir, join
import struct
from array import array

os.chdir('..')
import functions as ft

weights_path = r'/home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/mnist_mlp_weights_and_biases_locked'
print(f"Loading weights and biases from: {weights_path}")

try:
    # Load weights from .npy files
    w1 = np.load(join(weights_path, 'mnist_mlp_w1.npy'))
    w2 = np.load(join(weights_path, 'mnist_mlp_w2.npy'))
    w3 = np.load(join(weights_path, 'mnist_mlp_w3.npy'))
    w_out = np.load(join(weights_path, 'mnist_mlp_w4.npy'))

    # Load biases from .npy files
    b1 = np.load(join(weights_path, 'mnist_mlp_b1.npy'))
    b2 = np.load(join(weights_path, 'mnist_mlp_b2.npy'))
    b3 = np.load(join(weights_path, 'mnist_mlp_b3.npy'))
    b_out = np.load(join(weights_path, 'mnist_mlp_b4.npy'))

    # Ensure biases are 2D row vectors for broadcasting in the forward pass
    # Keras saves biases as 1D (N,), but NumPy math expects (1, N)
    b1 = b1.reshape(1, -1)
    b2 = b2.reshape(1, -1)
    b3 = b3.reshape(1, -1)
    b_out = b_out.reshape(1, -1)

    print("Weights and biases loaded successfully!")
    print(f"W1 shape: {w1.shape}, b1 shape: {b1.shape}")
    print(f"W2 shape: {w2.shape}, b2 shape: {b2.shape}")
    print(f"W3 shape: {w3.shape}, b3 shape: {b3.shape}")
    print(f"W_out (W4) shape: {w_out.shape}, b_out (b4) shape: {b_out.shape}")

except FileNotFoundError as e:
    print(f"Error loading weight/bias files: {e}. Please ensure '{weights_path}' exists and contains the CSV files.")
    exit()

# --- Apply the 'unlock' operation to the loaded weights ---

lock_key = [101, 202, 303, 404, 505, 606, 707, 808]

# Create a dictionary of weights to pass to the unlock function
locked_weights_dict = {
    'W1': w1,
    'W2': w2,
    'W3': w3,
    'W4': w_out # Map w_out to W4 for the unlock function
}

print("\nUnlocking weights...")
unlocked_weights = ft.unlock_weights(locked_weights_dict, lock_key)

# Update the global weight variables with the unlocked versions
w1 = unlocked_weights['W1']
w2 = unlocked_weights['W2']
w3 = unlocked_weights['W3']
w_out = unlocked_weights['W4'] # Update w_out with the unlocked W4

print("Weights unlocked successfully!")

# --- Load testing data using the MNISTDataLoader class ---
print("\nLoading MNIST test data from binary files...")
_training_images_filepath = ""
_training_labels_filepath = ""
_test_images_filepath = r'/home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/mnist_test_data/t10k-images.idx3-ubyte'
_test_labels_filepath = r'/home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/mnist_test_data/t10k-labels.idx1-ubyte'

try:
    mnist_data_loader = ft.MNISTDataLoader(
        _training_images_filepath, _training_labels_filepath,
        _test_images_filepath, _test_labels_filepath
    )
    x_test = mnist_data_loader.x_test
    y_test_one_hot = mnist_data_loader.y_test # Store one-hot encoded labels
    
    # For accuracy calculation, we need the integer labels, not one-hot
    y_test = np.argmax(y_test_one_hot, axis=1) 
    
    print("MNIST test data loaded and preprocessed.")

    print("\nPerforming forward pass with unlocked weights...")
    Y_pred = ft.forward_pass(x_test, w1, b1, w2, b2, w3, b3, w_out, b_out)

    # Decode predictions to class labels
    predicted_labels = np.argmax(Y_pred, axis=1)

    # --- Calculate accuracy ---
    def calculate_accuracy(predicted_labels, true_labels):
        correct_predictions = np.sum(predicted_labels == true_labels)
        total_predictions = len(true_labels)
        accuracy = correct_predictions / total_predictions * 100
        return accuracy

    # Compute and print testing accuracy
    testing_accuracy = calculate_accuracy(predicted_labels, y_test)
    print(f"\nTesting Accuracy (with unlocked weights): {testing_accuracy:.2f}%")

except FileNotFoundError as e:
    print(f"Error loading MNIST files: {e}. Please ensure the MNIST dataset binary files are in the correct directory.")
    print("Expected paths:")
    print(f"  Test Images: {_test_images_filepath}")
    print(f"  Test Labels: {_test_labels_filepath}")
    exit()
except Exception as e:
    print(f"An unexpected error occurred during data loading or prediction: {e}")
    exit()


Loading weights and biases from: /home/xilinx/jupyter_notebooks/ring_oscillator_puf/sw/pynq/mnist_mlp_test/mnist_mlp_weights_and_biases_locked
Weights and biases loaded successfully!
Weights and biases loaded successfully!
W1 shape: (784, 512), b1 shape: (1, 512)
W2 shape: (512, 56), b2 shape: (1, 56)
W3 shape: (56, 128), b3 shape: (1, 128)
W_out (W4) shape: (128, 10), b_out (b4) shape: (1, 10)

Unlocking weights...
Weights unlocked successfully!
Unlocked W1 shape: (784, 512)

Loading MNIST test data from binary files...
MNIST Test Data loaded and processed:
  x_test shape: (10000, 784), dtype: float32
  y_test shape: (10000, 10), dtype: float64
MNIST test data loaded and preprocessed.

Performing forward pass with unlocked weights...

Testing Accuracy (with unlocked weights): 97.93%
